In [1]:
import pandas as pd
import numpy as np
import pyreadstat

In [2]:
R9 = pd.read_csv('/work/lamlab/data/R9.csv')
R8 = pd.read_csv('/work/lamlab/data/R8.csv')
R7 = pd.read_csv('/work/lamlab/data/R7.csv')

/tmp/ipykernel_2779017/3810890782.py:1: DtypeWarning: Columns (11,12,13,14,68,296,318,335,339,341,349,351,372,374,387) have mixed types. Specify dtype option on import or set low_memory=False.
  R9 = pd.read_csv('/work/lamlab/data/R9.csv')
/tmp/ipykernel_2779017/3810890782.py:2: DtypeWarning: Columns (43,45,47,49,53,169,191,387) have mixed types. Specify dtype option on import or set low_memory=False.
  R8 = pd.read_csv('/work/lamlab/data/R8.csv')
/tmp/ipykernel_2779017/3810890782.py:3: DtypeWarning: Columns (9,10,11,12,13,15,253,282,311,313,334,336,349) have mixed types. Specify dtype option on import or set low_memory=False.
  R7 = pd.read_csv('/work/lamlab/data/R7.csv')


In [3]:
R9['ROUND'] = 9
R8['ROUND'] = 8
R7['ROUND'] = 7

columns = [
    'COUNTRY',
    'EA_GPS_LA',
    'EA_GPS_LO',
    'EA_SVC_A',
    'EA_SVC_B',
    'EA_SVC_C',
    'URBRUR',
    'REGION',
    'ROUND'
    ]

R9 = R9[columns]
R8 = R8[columns]
R7 = R7[columns]

R9 = R9.groupby(['EA_GPS_LA', 'EA_GPS_LO'], as_index=False).mean()
R8 = R8.groupby(['EA_GPS_LA', 'EA_GPS_LO'], as_index=False).mean()
R7 = R7.groupby(['EA_GPS_LA', 'EA_GPS_LO'], as_index=False).mean()

R9= R9[R9['EA_SVC_C'].isin([0, 1])]
R8= R8[R8['EA_SVC_C'].isin([0, 1])]
R7= R7[R7['EA_SVC_C'].isin([0, 1])]

In [4]:
combined_df = pd.concat([R7, R8, R9], ignore_index=True)
combined_df.to_csv('/work/lamlab/MERGED.csv', index=False)

## Structure for pretraining on R7, R8, and R9 

In [6]:
import os
import re
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Function to extract coordinates from file name
def extract_coordinates(file_path):
    file_name = os.path.basename(file_path)
    # Adjust the regex pattern if your file naming convention changes
    pattern = r"[a-z]*_image_([+-]?\d+\.\d+)_([+-]?\d+\.\d+)_\d{4}-\d{2}-\d{2}\.tif"
    match = re.match(pattern, file_name)
    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        return latitude, longitude
    else:
        raise ValueError(f"Filename '{file_name}' does not match the expected pattern.")

# Function to match coordinates in the DataFrame
def match_coordinates(df, latitude, longitude, tolerance=1e-5):
    matched_rows = df[
        (df['EA_GPS_LA'].between(latitude - tolerance, latitude + tolerance)) &
        (df['EA_GPS_LO'].between(longitude - tolerance, longitude + tolerance))
    ]
    return matched_rows

# Function to ensure directory exists
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Paths configuration
dataset_directory = "/work/lamlab/data"

csv_file_path = "/work/lamlab/data/MERGED.csv"

# Output directories
structured_dataset_b = "/work/lamlab/data/PIPEDWATER"
structured_dataset_c = "/work/lamlab/data/SEWAGE"

# Split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Load CSV
try:
    df = pd.read_csv(csv_file_path)
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error loading CSV file: {e}")

# Check if required columns exist
required_columns = {'EA_GPS_LA', 'EA_GPS_LO', 'EA_SVC_B', 'EA_SVC_C'}
if not required_columns.issubset(df.columns):
    print(f"CSV does not contain required columns: {required_columns}")

# Initialize a list to store matched file information
matched_files = []

# Iterate over all .tif files in the dataset directory
print("Scanning .tif files and matching coordinates...")
subfolders = ['R7', 'R8', 'R9']
for sub in subfolders:
    subdir_path = os.path.join(dataset_directory, sub)
    if not os.path.exists(subdir_path):
        print(f"Subfolder '{sub}' not found at {subdir_path}, skipping.")
        continue

    for root, dirs, files in os.walk(subdir_path):
        for file in tqdm(files, desc=f"Processing files in {sub}", unit="file"):
            if file.lower().endswith(".tif"):
                file_path = os.path.join(root, file)
                try:
                    lat, lon = extract_coordinates(file_path)
                except ValueError as ve:
                    print(f"Skipping file '{file_path}': {ve}")
                    continue

                matched = match_coordinates(df, lat, lon)

                if not matched.empty:
                    row = matched.iloc[0]
                    ea_svc_b = row['EA_SVC_B']
                    ea_svc_c = row['EA_SVC_C']
                    matched_files.append({
                        'file_path': file_path,
                        'EA_SVC_B': ea_svc_b,
                        'EA_SVC_C': ea_svc_c
                    })
                else:
                    print(f"No match found for file '{file_path}'.")

# Convert matched_files to a DataFrame
matched_df = pd.DataFrame(matched_files)
print(f"Total matched files: {len(matched_df)}")

# Check if there are matched files
if matched_df.empty:
    print("No matched files found. Exiting.")

# Save matched information to a CSV (optional)
matched_df = matched_df[((matched_df['EA_SVC_B']==0) | (matched_df['EA_SVC_B']==1)) & ((matched_df['EA_SVC_C']==0) | (matched_df['EA_SVC_C']==1))]
matched_df.to_csv("/work/lamlab/matched_files_labels.csv", index=False)
print("Matched file information saved to 'matched_files_labels.csv'.")

Error loading CSV file: [Errno 2] No such file or directory: '/work/lamlab/data/MERGED.csv'
Scanning .tif files and matching coordinates...


Processing files in R7: 0file [00:00, ?file/s]
Processing files in R7: 0file [00:00, ?file/s]


Processing files in R7: 100%|██████████| 3037/3037 [00:01<00:00, 1927.85file/s]
Processing files in R7: 0file [00:00, ?file/s]
Processing files in R7: 100%|██████████| 4692/4692 [00:03<00:00, 1483.23file/s]
Processing files in R8: 0file [00:00, ?file/s]
Processing files in R8: 0file [00:00, ?file/s]
Processing files in R8: 100%|██████████| 3821/3821 [00:02<00:00, 1419.48file/s]
Processing files in R8: 0file [00:00, ?file/s]
Processing files in R8: 100%|██████████| 5104/5104 [00:02<00:00, 1774.80file/s]
Processing files in R9: 0file [00:00, ?file/s]
Processing files in R9: 0file [00:00, ?file/s]
Processing files in R9: 100%|██████████| 3203/3203 [00:01<00:00, 1766.28file/s]
Processing files in R9: 0file [00:00, ?file/s]
Processing files in R9: 100%|██████████| 4176/4176 [00:02<00:00, 1643.74file/s]


Total matched files: 326654
Matched file information saved to 'matched_files_labels.csv'.


In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Splitting the data
print("Splitting data into train, validation, and test sets...")

# First split into train and temp (train: 80%, temp: 20%)
train_df, temp_df = train_test_split(
    matched_df,
    test_size=(val_ratio + test_ratio),
    stratify=matched_df['EA_SVC_C'],  # Stratify based on EA_SVC_C to maintain label distribution
    random_state=42
)

# Then split temp into validation and test (each 10%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=test_ratio / (val_ratio + test_ratio),
    stratify=temp_df['EA_SVC_C'],
    random_state=42
)

print(f"Training set: {len(train_df)} files")
print(f"Validation set: {len(val_df)} files")
print(f"Test set: {len(test_df)} files")

# Ensure the directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to copy a single file
def copy_file(row, split_name):
    # For EA_SVC_C
    label_b = row['EA_SVC_C']
    dest_dir_b = os.path.join(structured_dataset_c, split_name, str(label_b))
    ensure_dir(dest_dir_b)
    shutil.copy2(row['file_path'], dest_dir_b)

# Function to copy files for a split using ThreadPoolExecutor
def copy_files_parallel(df_split, split_name):
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        # Submit tasks for parallel execution
        futures = [executor.submit(copy_file, row, split_name) for _, row in df_split.iterrows()]
        # Track progress
        for _ in tqdm(as_completed(futures), desc=f"Processing {split_name} set", total=len(df_split)):
            pass

# Copy files for each split
print("Copying files to structured directories...")
for split_name, df_split in zip(['train', 'val', 'test'], [train_df, val_df, test_df]):
    print(f"Copying {split_name} set...")
    copy_files_parallel(df_split, split_name)

print("All files have been copied to their respective directories.")

Splitting data into train, validation, and test sets...


Training set: 259918 files
Validation set: 32490 files
Test set: 32490 files
Copying files to structured directories...
Copying train set...


Processing train set:  28%|██▊       | 73359/259918 [06:57<18:36, 167.04it/s]

In [10]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Splitting the data
print("Splitting data into train, validation, and test sets...")

# First split into train and temp (train: 80%, temp: 20%)
train_df, temp_df = train_test_split(
    matched_df,
    test_size=(val_ratio + test_ratio),
    stratify=matched_df['EA_SVC_B'],  # Stratify based on EA_SVC_B to maintain label distribution
    random_state=42
)

# Then split temp into validation and test (each 10%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=test_ratio / (val_ratio + test_ratio),
    stratify=temp_df['EA_SVC_B'],
    random_state=42
)

print(f"Training set: {len(train_df)} files")
print(f"Validation set: {len(val_df)} files")
print(f"Test set: {len(test_df)} files")

# Ensure the directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to copy a single file
def copy_file(row, split_name):
    # For EA_SVC_B
    label_b = row['EA_SVC_B']
    dest_dir_b = os.path.join(structured_dataset_b, split_name, str(label_b))
    ensure_dir(dest_dir_b)
    shutil.copy2(row['file_path'], dest_dir_b)

# Function to copy files for a split using ThreadPoolExecutor
def copy_files_parallel(df_split, split_name):
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        # Submit tasks for parallel execution
        futures = [executor.submit(copy_file, row, split_name) for _, row in df_split.iterrows()]
        # Track progress
        for _ in tqdm(as_completed(futures), desc=f"Processing {split_name} set", total=len(df_split)):
            pass

# Copy files for each split
print("Copying files to structured directories...")
for split_name, df_split in zip(['train', 'val', 'test'], [train_df, val_df, test_df]):
    print(f"Copying {split_name} set...")
    copy_files_parallel(df_split, split_name)

print("All files have been copied to their respective directories.")

Splitting data into train, validation, and test sets...
Training set: 259918 files
Validation set: 32490 files
Test set: 32490 files
Copying files to structured directories...
Copying train set...


Processing train set: 100%|██████████| 259918/259918 [19:58<00:00, 216.91it/s]


Copying val set...


Processing val set: 100%|██████████| 32490/32490 [02:22<00:00, 227.71it/s]


Copying test set...


Processing test set: 100%|██████████| 32490/32490 [02:22<00:00, 227.70it/s]

All files have been copied to their respective directories.


#### Separating Landsat and Sentinel

In [ ]:
import os
import shutil
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

def detect_sensor(filename):
    fn = filename.lower()
    if 'sentinel' in fn:
        return 'Sentinel'
    if 'landsat' in fn:
        return 'Landsat'
    return None

def find_split_and_class(root):
    """
    Finds the split (train/val/test) and class label (e.g. 0/1) from a path.
    Returns (split, class_label) or (None, None) if not found.
    """
    parts = [p for p in root.split(os.sep) if p]
    for s in ('train', 'val', 'test'):
        if s in parts:
            idx = parts.index(s)
            class_label = parts[idx+1] if idx+1 < len(parts) else ''
            return s, class_label
    return None, None

def create_dest_path(root, file, new_base):
    """
    Generate destination path preserving split/class structure and separating sensors.
    Example:
      /work/lamlab/PIPEDWATER/train/0/sentinel_image_...tif
      -> /work/lamlab/PIPEDWATER_by_sensor/Sentinel/train/0/sentinel_image_...tif
    """
    sensor = detect_sensor(file)
    if sensor is None:
        return None
    split, class_label = find_split_and_class(root)
    if split is None:
        return None
    return os.path.join(new_base, sensor, split, class_label, file)

def process_file_pair(args):
    src_path, dest_path = args
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    shutil.copy2(src_path, dest_path)
    return True

def process_dataset(dataset):
    """
    Process all .tif files under /work/lamlab/{dataset}/{train,val,test}/... and copy them to
    /work/lamlab/{dataset}_by_sensor/{Sentinel|Landsat}/{train,val,test}/{class}/...
    """
    original_base = f"/work/lamlab/{dataset}"
    new_base = f"/work/lamlab/{dataset}_by_sensor"

    if not os.path.isdir(original_base):
        print(f"Source folder not found: {original_base}, skipping.")
        return

    # Collect file pairs
    file_pairs = []
    for root, _, files in os.walk(original_base):
        for file in files:
            if not file.lower().endswith(".tif"):
                continue
            dest_path = create_dest_path(root, file, new_base)
            if dest_path:
                file_pairs.append((os.path.join(root, file), dest_path))
            else:
                # optional: uncomment next line to see skipped files
                # print(f"Skipping {os.path.join(root, file)} (no sensor or split/class detected)")
                pass

    if not file_pairs:
        print(f"No .tif files found for {dataset} (or none matched sensor+split structure).")
        return

    # Use all available CPUs; copying is IO-bound so a moderate number is fine
    max_workers = min(os.cpu_count() or 1, 8)

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_file_pair, pair) for pair in file_pairs]
        with tqdm(total=len(file_pairs), desc=f"Processing {dataset} files", unit="file", dynamic_ncols=True) as pbar:
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    tqdm.write(f"Error copying file: {e}")
                pbar.update(1)

def main():
    for dataset in ['PIPEDWATER', 'SEWAGE']:
        process_dataset(dataset)
    print("Dataset reorganization completed successfully!")

if __name__ == "__main__":
    main()

In [5]:
import os

print(len(os.listdir('/work/lamlab/PIPEDWATER_by_sensor/Sentinel/train/1.0')))
print(len(os.listdir('/work/lamlab/SEWAGE_by_sensor/Sentinel/train/1.0')))
print(len(os.listdir('/work/lamlab/PIPEDWATER_by_sensor/Landsat/train/1.0')))
print(len(os.listdir('/work/lamlab/SEWAGE_by_sensor/Landsat/train/1.0')))


84227


47375
59063
33572


## URBAN VS RURAL

In [19]:
#!/usr/bin/env python3
import os
import re
import shutil
import multiprocessing
import pandas as pd
from functools import partial
from tqdm import tqdm

# --- Parameters: update these paths as needed ---
BASE_DIR = "/work/lamlab/TEST_C/Sentinel"
CSV_PATH = "/dkucc/home/oe23/DATABASE/RAW/MERGED.csv"
# Only process the 'val' split
SPLITS = ["val"]

# --- Functions ---

def extract_coordinates(file_path):
    """
    Extracts coordinates from a file name matching pattern:
    <anything>_image_<latitude>_<longitude>_YYYY-MM-DD.tif
    """
    file_name = os.path.basename(file_path)
    pattern = r"[a-zA-Z0-9_]+_image_([+-]?\d+\.\d+)_([+-]?\d+\.\d+)_\d{4}-\d{2}-\d{2}\.tif"
    match = re.match(pattern, file_name)
    if match:
        return float(match.group(1)), float(match.group(2))
    raise ValueError(f"Filename '{file_name}' doesn't match expected pattern.")

def match_coordinates(df, latitude, longitude, tolerance=1e-5):
    """
    Finds the row in the DataFrame where EA_GPS_LA and EA_GPS_LO are within a tolerance.
    """
    matched = df[
        (df['EA_GPS_LA'].between(latitude - tolerance, latitude + tolerance)) &
        (df['EA_GPS_LO'].between(longitude - tolerance, longitude + tolerance))
    ]
    if matched.empty:
        raise ValueError(f"No match found in CSV for coordinates ({latitude}, {longitude}).")
    return matched.iloc[0]  # Return the first match

def process_file(file_info, df, base_dir):
    """
    Processes one file:
      - file_info: (split, class_label, full_path)
      - Extracts coordinates from filename.
      - Matches them in df to get URBRUR value.
      - Determines the destination folder based on the 'val' split and the URBRUR value.
      - Copies the file to the new destination while preserving the class (0 or 1) subfolder.
    """
    split, class_label, file_path = file_info
    try:
        # Extract coordinates from filename
        lat, lon = extract_coordinates(file_path)
        # Match coordinates in DataFrame to retrieve the row and its URBRUR value
        row = match_coordinates(df, lat, lon)
        urbrur_value = row['URBRUR']
        
        # Create a destination folder name based on the URBRUR value.
        # For instance, if URBRUR is 1.0, destination becomes "val_1.0"
        # If you prefer a different naming convention (e.g., removing the decimal),
        # you can adjust the following line.
        dest_split = f"{split}_{urbrur_value}"
        
        # Build the full destination directory path, preserving the class (0 or 1)
        dest_dir = os.path.join(base_dir, dest_split, class_label)
        os.makedirs(dest_dir, exist_ok=True)

        # Copy the file to the destination folder
        dest_path = os.path.join(dest_dir, os.path.basename(file_path))
        shutil.copy2(file_path, dest_path)
    except Exception as e:
        # Log errors; you could also write to a log file if preferred
        print(f"Error processing file {file_path}: {e}")

def gather_files(base_dir, splits):
    """
    Gathers all .tif file paths from the specified splits.
    Returns a list of tuples: (split, class_label, file_path).
    """
    file_list = []
    for split in splits:
        split_dir = os.path.join(base_dir, split)
        if not os.path.isdir(split_dir):
            print(f"Warning: {split_dir} does not exist, skipping...")
            continue
        for class_label in ["0", "1"]:
            class_dir = os.path.join(split_dir, class_label)
            if not os.path.isdir(class_dir):
                print(f"Warning: {class_dir} does not exist, skipping...")
                continue
            for fname in os.listdir(class_dir):
                if fname.lower().endswith(".tif"):
                    file_path = os.path.join(class_dir, fname)
                    file_list.append((split, class_label, file_path))
    return file_list

def main():
    # Load the CSV file into a DataFrame
    df = pd.read_csv(CSV_PATH)
    # Optionally, you can inspect the DataFrame's head:
    # print(df.head())

    # Gather all .tif files from the 'val' split
    files_to_process = gather_files(BASE_DIR, SPLITS)
    print(f"Found {len(files_to_process)} files in the val/ split to process.")

    # Use all available CPUs
    cpus = multiprocessing.cpu_count()
    print(f"Using {cpus} CPUs for processing.")

    # Prepare the worker function with additional parameters using partial
    worker = partial(process_file, df=df, base_dir=BASE_DIR)
    
    # Use multiprocessing Pool with tqdm progress bar
    with multiprocessing.Pool(processes=cpus) as pool:
        # tqdm will display progress as files are processed.
        for _ in tqdm(pool.imap_unordered(worker, files_to_process), total=len(files_to_process)):
            pass

if __name__ == "__main__":
    main()

Found 18631 files in the val/ split to process.
Using 94 CPUs for processing.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18631/18631 [01:51<00:00, 167.08it/s]
